In [ ]:
import requests
import json

# 요청 헤더 설정 (웹사이트에서 차단되지 않도록 user-agent 추가)
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'
}

class CardCrawler:
    def __init__(self, card_type):
        """
        CardCrawler 클래스 초기화
        :param card_type: 카드 타입 (CRD: 신용카드, CHK: 체크카드)
        """
        self.card_type = card_type  # 카드 타입 저장
        self.page_num = 1  # 크롤링할 페이지 번호 초기화
        self.cards_data = []  # 크롤링된 카드 정보를 저장할 리스트

    def get_card_data(self):
        """
        카드탭 페이지를 API로부터 받아오는 함수
        :return: 카드 기본 정보를 JSON 데이터로 반환 (요청 실패 시 None 반환)
        """
        url = f'https://api.card-gorilla.com:8080/v1/cards?p={self.page_num}&perPage=10&cate={self.card_type}'
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            return response.json()  # JSON 데이터를 반환
        else:
            print(f"Page {self.page_num} 요청 실패! 상태 코드: {response.status_code}")
            return None  # 요청 실패 시 None 반환

    def print_card_details(self, card):
        """
        카드의 세부 정보를 출력하는 함수
        :param card: 개별 카드 정보 (딕셔너리 형태)
        :return: 카드별 API URL 반환 (없을 경우 None)
        """
        if card.get('is_discon', False):  # 카드가 단종된 경우 출력하지 않고 건너뜀
            print(f"  카드 '{card.get('name')}'은(는) 사용 불가 카드입니다. 건너뜁니다.")
            return None
        
        # 카드 기본 정보 출력
        card_name = card.get('name', '이름 없음')
        card_idx = card.get('idx')  # 카드의 고유 인덱스
        card_cate_txt = card.get('cate_txt', '카테고리 없음')
        card_corp_txt = card.get('corp_txt', '발급사 없음')
        
        print(f"  카드 이름: {card_name}")
        print(f"    카드 카테고리: {card_cate_txt}")
        print(f"    카드 발급사: {card_corp_txt}")
        
        if card_idx:
            # 카드 상세 정보 API URL 및 웹페이지 URL 생성
            card_detail_api_url = f"https://api.card-gorilla.com:8080/v1/cards/{card_idx}" 
            card_detail_page_url = f"https://www.card-gorilla.com/card/detail/{card_idx}"
            
            print(f"    카드 상세 API URL: {card_detail_api_url}")
            print(f"    카드 상세 페이지 URL: {card_detail_page_url}")
            
            return card_detail_api_url  # 카드 상세 API URL 반환
        return None

    def get_card_detail(self, card_detail_url):
        """
        카드의 세부 정보를 받아오는 함수
        :param card_detail_url: 카드 상세 정보 API URL
        :return: 카드 상세 정보 JSON 데이터 반환 (요청 실패 시 None 반환)
        """
        response = requests.get(card_detail_url, headers=headers)
        
        if response.status_code == 200:
            return response.json()  # 카드 상세 정보를를 JSON 데이터로로 반환
        else:
            print(f"    카드 상세 정보 요청 실패! 상태 코드: {response.status_code}")
            return None

    def print_benefits(self, key_benefit):
        """
        카드 혜택을 출력하는 함수
        :param key_benefit: 카드 혜택 리스트
        :return: 카드 혜택 목록 반환 (없을 경우 기본 메시지 반환)
        """
        comments = [item['comment'] for item in key_benefit[:-1]]  # 마지막 항목 제외
        return comments if comments else ["혜택 정보가 없습니다."]

    def crawl_cards(self):
        """
        카드 크롤링을 수행하는 메인 함수
        """
        while True:
            json_data = self.get_card_data()  # get_card_data()함수를 호출하여여 카드탭 API에서 카드 데이터 요청
            
            if json_data:
                data_list = json_data.get('data', [])  # 기본 카드 데이터 리스트 추출
                
                if data_list:
                    print(f"\nPage {self.page_num}:") # 현재 페이지 출력
                    for card in data_list:
                        card_detail_url = self.print_card_details(card)  # print_card_details를 활용해 카드별 api 추출하여 저장
                        
                        if card_detail_url:  # 사용 가능한 카드만 처리
                            card_data = self.get_card_detail(card_detail_url) # get_card_detail을 활용하여 카드별별 api를 json 형태로 추출하여 저장
                            
                            if card_data: # card_data가 None이 아니라면 (즉, API 요청이 성공했다면), 아래의 코드를 실행
                                key_benefit = card_data.get('key_benefit', [])  # 카드 상세 정보에서 'key_benefit' 필드를 가져옴 (없으면 빈 리스트 반환)
                                benefits = self.print_benefits(key_benefit) # 가져온 혜택 정보를 정리하는 print_benefits 함수 호출
                                
                                # 카드 정보를 딕셔너리 형태로 정리하여 저장
                                card_idx = card.get('idx')
                                card_page_url = f"https://www.card-gorilla.com/card/detail/{card_idx}" if card_idx else "URL 없음"
                                card_info = {
                                    'name': card.get('name', '이름 없음'),
                                    'cate_txt': card.get('cate_txt', '카테고리 없음'),
                                    'corp_txt': card.get('corp_txt', '발급사 없음'),
                                    'benefits': benefits,
                                    'page_url': card_page_url
                                }
                                self.cards_data.append(card_info)
                    
                    self.page_num += 1  # 다음 페이지로 이동
                else:
                    print(f"더 이상 카드 정보가 없습니다. 크롤링을 종료합니다.")
                    break
            else:
                break  # 요청 실패 시 종료

        # 크롤링이 끝난 후 JSON 파일로 저장
        self.save_to_json()

    def save_to_json(self):
        """
        크롤링된 카드 정보를 JSON 파일로 저장하는 함수
        """
        with open(f'{self.card_type}_cards_data.json', 'w', encoding='utf-8') as json_file:
            json.dump(self.cards_data, json_file, ensure_ascii=False, indent=4)
        print(f"{self.card_type}_cards_data.json 파일로 저장되었습니다.")


# 이 파일이 직접 실행될 때만 아래 코드를 실행하도록 설정
# 다른 파일에서 이 파일을 임포트하면 아래 코드는 실행되지 않음
if __name__ == "__main__":
    # 사용자에게 카드 타입 입력 받기
    card_type = input("신용카드 (CRD) 또는 체크카드 (CHK)를 선택하세요: ").strip().upper()
    
    if card_type not in ['CRD', 'CHK']:
        print("잘못된 카드 타입입니다. 'CRD' 또는 'CHK'를 입력하세요.")
    else:
        card_crawler = CardCrawler(card_type)  # CardCrawler 객체 생성
        card_crawler.crawl_cards()  # 카드 크롤링 시작



Page 1:
  카드 이름: 신한카드 Mr.Life
    카드 카테고리: 신용
    카드 발급사: 신한카드
    카드 상세 API URL: https://api.card-gorilla.com:8080/v1/cards/13
    카드 상세 페이지 URL: https://www.card-gorilla.com/card/detail/13
  카드 이름: 삼성카드 taptap O
    카드 카테고리: 신용
    카드 발급사: 삼성카드
    카드 상세 API URL: https://api.card-gorilla.com:8080/v1/cards/51
    카드 상세 페이지 URL: https://www.card-gorilla.com/card/detail/51
  카드 이름: 삼성카드 & MILEAGE PLATINUM (스카이패스)
    카드 카테고리: 신용
    카드 발급사: 삼성카드
    카드 상세 API URL: https://api.card-gorilla.com:8080/v1/cards/49
    카드 상세 페이지 URL: https://www.card-gorilla.com/card/detail/49
  카드 이름: 현대카드 M
    카드 카테고리: 신용
    카드 발급사: 현대카드
    카드 상세 API URL: https://api.card-gorilla.com:8080/v1/cards/2669
    카드 상세 페이지 URL: https://www.card-gorilla.com/card/detail/2669
  카드 이름: KB국민 My WE:SH 카드
    카드 카테고리: 신용
    카드 발급사: KB국민카드
    카드 상세 API URL: https://api.card-gorilla.com:8080/v1/cards/2441
    카드 상세 페이지 URL: https://www.card-gorilla.com/card/detail/2441
  카드 이름: LOCA LIKIT 1.2
    카드 카테고리: 신용
    카드 발